## RAG With Multi Data Source

make wrapper for every tool, tools example : wikipedia, arxiv

after combining all the the tools we will make a chain with user and llm

## Wikipedia Tool

In [2]:
# wrapper that helps wikipedia to run the query
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [3]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)
print(wiki.name)

wikipedia


## normal web retriever


In [4]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs= loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
embeddings = OllamaEmbeddings(
    model="llama3.2:latest",
)
vectordb = FAISS.from_documents(documents,embedding=embeddings)
retriever = vectordb.as_retriever
retriever


USER_AGENT environment variable not set, consider setting it to identify your requests.


<bound method VectorStore.as_retriever of <langchain_community.vectorstores.faiss.FAISS object at 0x114753ed0>>

now we'll make a tool over this retrieval, basically whatever you have retrieved you'll make a tool over it

In [5]:
from langchain.tools.retriever import create_retriever_tool

retrieval_tool = create_retriever_tool(retriever,"langsmith_search","Search for information about Langsmith")


In [6]:
retrieval_tool.name

'langsmith_search'

## Arxiv Tool

In [7]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun


arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [8]:
tools=[wiki,arxiv,retrieval_tool]

In [9]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/dishaarora/Desktop/langchain/venv/lib/python3.11/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith_search', description='Search for information about Langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x1069365c0>, retriever=<bound method VectorStore.as_retriever of <langchain_community.vectorstores.faiss.FAISS object at 0x114753ed0>>, document_prompt=PromptTemplate

In [13]:
from langchain_ollama.llms import OllamaLLM



llm = OllamaLLM(model="llama3.2:latest")



In [12]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

/Users/dishaarora/Desktop/langchain/venv/lib/python3.11/site-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [15]:
from langchain.agents import initialize_agent
from langchain.agents.agent_types import AgentType

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Example query to the agent
response = agent.run("Find me some papers on quantum computing and summarize them.")
print(response)



> Entering new AgentExecutor chain...


/var/folders/nw/9f0gp_2n35qd10hyfk73f59c0000gn/T/ipykernel_50158/3525505534.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. See LangGraph documentation for more details: https://langchain-ai.github.io/langgraph/. Refer here for its pre-built ReAct agent: https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/
  agent = initialize_agent(
/var/folders/nw/9f0gp_2n35qd10hyfk73f59c0000gn/T/ipykernel_50158/3525505534.py:12: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run("Find me some papers on quantum computing and summarize them.")


ValueError: Missing some input keys: {'page_content', 'input'}